In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sqlite3 as sq
data = pd.read_csv("final_data.csv",delimiter=',',parse_dates=True,encoding='UTF-8')

In [2]:
data.head()

,Rank,Time,Name,Country,Date of Birth,Place,City,Date,Gender,Event,Age
0,1,01:40.9,David Rudisha,KEN,12/17/88,1.0,London,9/8/12,Men,800 m,24
1,2,01:41.0,David Rudisha,KEN,12/17/88,1.0,Rieti,8/29/10,Men,800 m,22
2,3,01:41.1,David Rudisha,KEN,12/17/88,1.0,Berlin,8/22/10,Men,800 m,22
3,4,01:41.1,Wilson Kipketer,DEN,12/12/70,1.0,Koln,8/24/97,Men,800 m,27
4,5,01:41.2,Wilson Kipketer,DEN,12/12/70,1.0,Zurich,8/13/97,Men,800 m,27


In [3]:
data['Event'].unique()

array(['800 m', '400 m', '10,000 m', 'Marathon', '100 m', 'Half marathon',
       '200 m', '1500 m', '5000 m'], dtype=object)

In [4]:
bins = [15,20,25,30,35,40,45,50,60]
labels =['15-19','20-24','25-29','30-34','35-39','40-44','45-49','50+']
data['AgeGroup'] = pd.cut(data['Age'], bins=bins, labels=labels)
data.head()

,Rank,Time,Name,Country,Date of Birth,Place,City,Date,Gender,Event,Age,AgeGroup
0,1,01:40.9,David Rudisha,KEN,12/17/88,1.0,London,9/8/12,Men,800 m,24,20-24
1,2,01:41.0,David Rudisha,KEN,12/17/88,1.0,Rieti,8/29/10,Men,800 m,22,20-24
2,3,01:41.1,David Rudisha,KEN,12/17/88,1.0,Berlin,8/22/10,Men,800 m,22,20-24
3,4,01:41.1,Wilson Kipketer,DEN,12/12/70,1.0,Koln,8/24/97,Men,800 m,27,25-29
4,5,01:41.2,Wilson Kipketer,DEN,12/12/70,1.0,Zurich,8/13/97,Men,800 m,27,25-29


In [5]:
data['Event'] = np.where(data['Event'] == '100 m', '100m',data['Event'])
data['Event'] = np.where(data['Event'] == '200 m', '200m',data['Event'])
data['Event'] = np.where(data['Event'] == '400 m', '400m',data['Event'])
data['Event'] = np.where(data['Event'] == '800 m', '800m',data['Event'])
data['Event'] = np.where(data['Event'] == '1500 m', '1500m',data['Event'])
data['Event'] = np.where(data['Event'] == '5000 m', '5000m',data['Event'])
data['Event'] = np.where(data['Event'] == '10,000 m', '10000m',data['Event'])
data['Event'] = np.where(data['Event'] == 'Half marathon', 'Halfmarathon',data['Event'])

In [6]:
country_event = data.groupby(['Country','Event','Gender'], as_index=False).count()
country_event_df = country_event[['Country','Event','Gender','Rank']]
country_event_count_df = country_event_df.rename(columns={"Rank":"Count"})
country_event_count_df[(country_event_count_df['Country']=='USA') & (country_event_count_df['Event']=='800 m')]
country_event_count_df.head()

,Country,Event,Gender,Count
0,AHO,100m,Men,6
1,AHO,200m,Men,3
2,AIA,200m,Men,1
3,ALG,1500m,Men,90
4,ALG,1500m,Women,10


In [7]:
def event_data():
    df2 = country_event_count_df[(country_event_count_df['Country']=="KEN") & (country_event_count_df['Event']=="Marathon")]
    df3 = df2[(df2['Event']=="100 m")]
  

    return df2
print(event_data())

    Country     Event Gender  Count
332     KEN  Marathon    Men    576
333     KEN  Marathon  Women    222


In [10]:
from sqlalchemy import create_engine

In [11]:
sql_data = 'db/rundata.sqlite'
conn = sq.connect(sql_data)

In [12]:
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS rundata''')

In [13]:
data.to_sql('rundata', conn, if_exists='replace', index=False)

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [14]:
data = pd.read_sql('select * from rundata', conn)
data.head()

,Rank,Time,Name,Country,Date of Birth,Place,City,Date,Gender,Event,Age,AgeGroup
0,1,01:40.9,David Rudisha,KEN,12/17/88,1.0,London,9/8/12,Men,800m,24,20-24
1,2,01:41.0,David Rudisha,KEN,12/17/88,1.0,Rieti,8/29/10,Men,800m,22,20-24
2,3,01:41.1,David Rudisha,KEN,12/17/88,1.0,Berlin,8/22/10,Men,800m,22,20-24
3,4,01:41.1,Wilson Kipketer,DEN,12/12/70,1.0,Koln,8/24/97,Men,800m,27,25-29
4,5,01:41.2,Wilson Kipketer,DEN,12/12/70,1.0,Zurich,8/13/97,Men,800m,27,25-29


In [15]:
gender_df = data.groupby(['Country','Gender'], as_index=False)
gender_count = gender_df.count()
gender_count_df = gender_count[['Country','Gender','Rank']]
gender_count_renamed= gender_count_df.rename(columns={"Rank":"Count"})

In [25]:
def gender_data():
    df2 = genderData[genderData['Country']=='KEN']
    data2 = {
        "country": df2.Country.values.tolist(),
        "gender": df2.Gender.values.tolist(),
        "noOfAthletes":df2.Count.values.tolist()
    }

    return  data2
print(gender_data())

{'country': ['KEN', 'KEN'], 'gender': ['Men', 'Women'], 'noOfAthletes': [3006, 1220]}


In [19]:
gender_combined = data.groupby(['AgeGroup','Country','Gender'], as_index=False)
gender_combined_count = gender_combined.count()
gender_combined_count_df = gender_combined_count[['Country','AgeGroup','Gender','Rank']]
gender_combined_count_df = gender_combined_count_df.rename(columns={"Rank":"Count"})
gender_combined_count_df[gender_combined_count_df['Country']=='USA']

,Country,AgeGroup,Gender,Count
63,USA,15-19,Men,90
64,USA,15-19,Women,46
204,USA,20-24,Men,876
205,USA,20-24,Women,503
338,USA,25-29,Men,626
339,USA,25-29,Women,596
434,USA,30-34,Men,182
435,USA,30-34,Women,219
474,USA,35-39,Men,12
475,USA,35-39,Women,22


In [21]:
def agegroup_data():
    df4 = gender_combined_count_df[gender_combined_count_df['Country']=="KEN"]
    data4 = {
        "country": df4.Country.values.tolist(),
        "agegroup": df4.AgeGroup.values.tolist(),
        "gender": df4.Gender.values.tolist(),
        "noOfAthletes":df4.Count.values.tolist()
    }

    return data4
print(agegroup_data())

{'country': ['KEN', 'KEN', 'KEN', 'KEN', 'KEN', 'KEN', 'KEN', 'KEN', 'KEN', 'KEN', 'KEN'], 'agegroup': ['15-19', '15-19', '20-24', '20-24', '25-29', '25-29', '30-34', '30-34', '35-39', '35-39', '40-44'], 'gender': ['Men', 'Women', 'Men', 'Women', 'Men', 'Women', 'Men', 'Women', 'Men', 'Women', 'Women'], 'noOfAthletes': [379, 98, 1258, 382, 1047, 491, 299, 205, 22, 40, 2]}


In [22]:
men_women_df = data.groupby(['AgeGroup','Country','Gender'], as_index=False)
men_women_count = men_women_df.count()
men_women_count_df = men_women_count[['AgeGroup','Country','Gender','Rank']]
men_women_count_renamed = men_women_count_df.rename(columns={"Rank":"Count"})

women_filtered = men_women_count_renamed[men_women_count_renamed['Gender']=='Women']
women_filtered.head()

,AgeGroup,Country,Gender,Count
5,15-19,AZE,Women,1
7,15-19,BAH,Women,2
9,15-19,BDI,Women,4
13,15-19,BRN,Women,1
14,15-19,BUL,Women,3


In [23]:
men_filtered = men_women_count_renamed[men_women_count_renamed['Gender']=='Men']
#men_filtered.head()

In [24]:
gender_combined_count_df.to_sql('combined_age_data', conn, if_exists='replace', index=False)
gender_count_renamed.to_sql('gender_total_data', conn, if_exists='replace', index=False)
women_filtered.to_sql('women_age_data', conn, if_exists='replace', index=False)
men_filtered.to_sql('men_age_data', conn, if_exists='replace', index=False)
country_event_count_df.to_sql('country_event_data',conn,if_exists='replace',index=False)

In [25]:
conn.commit()
conn.close()